<a href="https://colab.research.google.com/github/panchambanerjee/code-carbon-expts/blob/main/codecarbon_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets accelerate huggingface_hub codecarbon -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.8/175.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import gc

In [3]:
from huggingface_hub import login
login()

### BERT Base Cased

In [4]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

# Compute intensive code goes here

ds = load_dataset("imdb")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

small_train_dataset = ds["train"].shuffle(seed=42).select(range(1000)).map(tokenize_function, batched=True)
small_eval_dataset = ds["test"].shuffle(seed=42).select(range(1000)).map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="/content/code-carbon-classification",
    num_train_epochs=2,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset
)

trainer.train()

# Compute intensive code stops here

tracker.stop()

[codecarbon INFO @ 07:00:01] [setup] RAM Tracking...
[codecarbon INFO @ 07:00:01] [setup] GPU Tracking...
[codecarbon INFO @ 07:00:01] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 07:00:01] [setup] CPU Tracking...
[codecarbon WARNING @ 07:00:01] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 07:00:03] We saw that you have a Intel(R) Xeon(R) CPU @ 2.30GHz but we don't know it. Please contact us.
[codecarbon INFO @ 07:00:03] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 07:00:03] >>> Tracker's metadata:
[codecarbon INFO @ 07:00:03]   Platform system: Linux-5.15.109+-x86_64-with-glibc2.35
[codecarbon INFO @ 07:00:03]   Python version: 3.10.6
[codecarbon INFO @ 07:00:03]   CodeCarbon version: 2.2.7
[codecarbon INFO @ 07:00:03]   Available RAM : 12.678 GB
[codecarbon INFO @ 07:00:03]   CPU count: 2
[codecarbon INFO @ 07:00:03]   CPU model: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 07:00:03]   GPU 

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

[codecarbon INFO @ 07:00:18] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.754395008087158 W
[codecarbon INFO @ 07:00:18] Energy consumed for all GPUs : 0.000041 kWh. Total GPU Power : 9.768 W
[codecarbon INFO @ 07:00:18] Energy consumed for all CPUs : 0.000178 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 07:00:18] 0.000238 kWh of electricity used since the beginning.


Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

[codecarbon INFO @ 07:00:33] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.754395008087158 W
[codecarbon INFO @ 07:00:33] Energy consumed for all GPUs : 0.000082 kWh. Total GPU Power : 9.967 W
[codecarbon INFO @ 07:00:33] Energy consumed for all CPUs : 0.000355 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 07:00:33] 0.000477 kWh of electricity used since the beginning.


...

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

[codecarbon INFO @ 07:01:11] [setup] RAM Tracking...
[codecarbon INFO @ 07:01:11] [setup] GPU Tracking...
[codecarbon INFO @ 07:01:11] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 07:01:11] [setup] CPU Tracking...
[codecarbon WARNING @ 07:01:11] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 07:01:13] We saw that you have a Intel(R) Xeon(R) CPU @ 2.30GHz but we don't know it. Please contact us.
[codecarbon INFO @ 07:01:13] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 07:01:13] >>> Tracker's metadata:
[codecarbon INFO @ 07:01:13]   Platform system: Linux-5.15.109+-x86_64-with-glibc2.35
[codecarbon INFO @ 07:01:13]   Python version: 3.10.6
[codecarbon INFO @ 07:01:13]   CodeCarbon version: 2.2.7
[codecarbon INFO @ 07:01:13]   Available RAM : 12.678 GB
[codecarbon INFO @ 07:01:13]   CPU count: 2
[codecarbon INFO @ 07:01:13]   CPU model: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 07:01:13]   GPU 

Step,Training Loss


[codecarbon INFO @ 07:01:18] Energy consumed for RAM : 0.000099 kWh. RAM Power : 4.754395008087158 W
[codecarbon INFO @ 07:01:18] Energy consumed for all GPUs : 0.000422 kWh. Total GPU Power : 61.984 W
[codecarbon INFO @ 07:01:18] Energy consumed for all CPUs : 0.000886 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 07:01:18] 0.001407 kWh of electricity used since the beginning.
[codecarbon INFO @ 07:01:28] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.754395008087158 W
[codecarbon INFO @ 07:01:28] Energy consumed for all GPUs : 0.000294 kWh. Total GPU Power : 70.444 W
[codecarbon INFO @ 07:01:28] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 07:01:28] 0.000491 kWh of electricity used since the beginning.
[codecarbon INFO @ 07:01:33] Energy consumed for RAM : 0.000119 kWh. RAM Power : 4.754395008087158 W
[codecarbon INFO @ 07:01:33] Energy consumed for all GPUs : 0.000700 kWh. Total GPU Power : 66.865 W
[codecarbon INFO @ 07:01:33] E

0.0032634881166092567

### BERT Base Uncased

In [5]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

# Compute intensive code goes here

ds = load_dataset("imdb")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

small_train_dataset = ds["train"].shuffle(seed=42).select(range(1000)).map(tokenize_function, batched=True)
small_eval_dataset = ds["test"].shuffle(seed=42).select(range(1000)).map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="/content/code-carbon-classification",
    num_train_epochs=2,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset
)

trainer.train()

# Compute intensive code stops here

tracker.stop()

[codecarbon INFO @ 07:04:21] [setup] RAM Tracking...
[codecarbon INFO @ 07:04:21] [setup] GPU Tracking...
[codecarbon INFO @ 07:04:21] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 07:04:21] [setup] CPU Tracking...
[codecarbon WARNING @ 07:04:21] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 07:04:22] We saw that you have a Intel(R) Xeon(R) CPU @ 2.30GHz but we don't know it. Please contact us.
[codecarbon INFO @ 07:04:22] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 07:04:22] >>> Tracker's metadata:
[codecarbon INFO @ 07:04:22]   Platform system: Linux-5.15.109+-x86_64-with-glibc2.35
[codecarbon INFO @ 07:04:22]   Python version: 3.10.6
[codecarbon INFO @ 07:04:22]   CodeCarbon version: 2.2.7
[codecarbon INFO @ 07:04:22]   Available RAM : 12.678 GB
[codecarbon INFO @ 07:04:22]   CPU count: 2
[codecarbon INFO @ 07:04:22]   CPU model: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 07:04:22]   GPU 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

[codecarbon INFO @ 07:04:31] [setup] RAM Tracking...
[codecarbon INFO @ 07:04:31] [setup] GPU Tracking...
[codecarbon INFO @ 07:04:31] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 07:04:31] [setup] CPU Tracking...
[codecarbon WARNING @ 07:04:31] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 07:04:33] We saw that you have a Intel(R) Xeon(R) CPU @ 2.30GHz but we don't know it. Please contact us.
[codecarbon INFO @ 07:04:33] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 07:04:33] >>> Tracker's metadata:
[codecarbon INFO @ 07:04:33]   Platform system: Linux-5.15.109+-x86_64-with-glibc2.35
[codecarbon INFO @ 07:04:33]   Python version: 3.10.6
[codecarbon INFO @ 07:04:33]   CodeCarbon version: 2.2.7
[codecarbon INFO @ 07:04:33]   Available RAM : 12.678 GB
[codecarbon INFO @ 07:04:33]   CPU count: 2
[codecarbon INFO @ 07:04:33]   CPU model: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 07:04:33]   GPU 

Step,Training Loss


[codecarbon INFO @ 07:04:38] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.754395008087158 W
[codecarbon INFO @ 07:04:38] Energy consumed for all GPUs : 0.000285 kWh. Total GPU Power : 68.375 W
[codecarbon INFO @ 07:04:38] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 07:04:38] 0.000482 kWh of electricity used since the beginning.
[codecarbon INFO @ 07:04:48] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.754395008087158 W
[codecarbon INFO @ 07:04:48] Energy consumed for all GPUs : 0.000297 kWh. Total GPU Power : 71.183 W
[codecarbon INFO @ 07:04:48] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 07:04:48] 0.000494 kWh of electricity used since the beginning.
[codecarbon INFO @ 07:04:53] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.754395008087158 W
[codecarbon INFO @ 07:04:53] Energy consumed for all GPUs : 0.000582 kWh. Total GPU Power : 71.476 W
[codecarbon INFO @ 07:04:53] E

0.0029819696499385445

### BERT Large Uncased

In [3]:
import gc
gc.collect()

60

In [4]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

# Compute intensive code goes here

ds = load_dataset("imdb")
model = AutoModelForSequenceClassification.from_pretrained("bert-large-uncased",\
                                                           num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

small_train_dataset = ds["train"].shuffle(seed=42).select(range(1000)).map(tokenize_function, batched=True)
small_eval_dataset = ds["test"].shuffle(seed=42).select(range(1000)).map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="/content/code-carbon-classification",
    num_train_epochs=2,
    push_to_hub=False,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset
)

trainer.train()

# Compute intensive code stops here

tracker.stop()

[codecarbon INFO @ 07:29:49] [setup] RAM Tracking...
[codecarbon INFO @ 07:29:49] [setup] GPU Tracking...
[codecarbon INFO @ 07:29:49] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 07:29:49] [setup] CPU Tracking...
[codecarbon WARNING @ 07:29:49] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 07:29:51] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 07:29:51] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 07:29:51] >>> Tracker's metadata:
[codecarbon INFO @ 07:29:51]   Platform system: Linux-5.15.109+-x86_64-with-glibc2.35
[codecarbon INFO @ 07:29:51]   Python version: 3.10.6
[codecarbon INFO @ 07:29:51]   CodeCarbon version: 2.2.7
[codecarbon INFO @ 07:29:51]   Available RAM : 12.678 GB
[codecarbon INFO @ 07:29:51]   CPU count: 2
[codecarbon INFO @ 07:29:51]   CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 07:29:51]   GPU 

[codecarbon INFO @ 07:30:06] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.754395008087158 W
[codecarbon INFO @ 07:30:06] Energy consumed for all GPUs : 0.000042 kWh. Total GPU Power : 10.15 W
[codecarbon INFO @ 07:30:06] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 07:30:06] 0.000239 kWh of electricity used since the beginning.


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

[codecarbon INFO @ 07:30:21] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.754395008087158 W
[codecarbon INFO @ 07:30:21] Energy consumed for all GPUs : 0.000085 kWh. Total GPU Power : 10.25 W
[codecarbon INFO @ 07:30:21] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 07:30:21] 0.000479 kWh of electricity used since the beginning.


...

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

[codecarbon INFO @ 07:30:51] Energy consumed for RAM : 0.000079 kWh. RAM Power : 4.754395008087158 W
[codecarbon INFO @ 07:30:51] Energy consumed for all GPUs : 0.000237 kWh. Total GPU Power : 26.826000000000004 W
[codecarbon INFO @ 07:30:51] Energy consumed for all CPUs : 0.000708 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 07:30:51] 0.001024 kWh of electricity used since the beginning.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.715100


[codecarbon INFO @ 07:31:06] Energy consumed for RAM : 0.000099 kWh. RAM Power : 4.754395008087158 W
[codecarbon INFO @ 07:31:06] Energy consumed for all GPUs : 0.000508 kWh. Total GPU Power : 65.022 W
[codecarbon INFO @ 07:31:06] Energy consumed for all CPUs : 0.000885 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 07:31:06] 0.001492 kWh of electricity used since the beginning.
[codecarbon INFO @ 07:31:21] Energy consumed for RAM : 0.000119 kWh. RAM Power : 4.754395008087158 W
[codecarbon INFO @ 07:31:21] Energy consumed for all GPUs : 0.000798 kWh. Total GPU Power : 69.63 W
[codecarbon INFO @ 07:31:21] Energy consumed for all CPUs : 0.001062 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 07:31:21] 0.001979 kWh of electricity used since the beginning.
[codecarbon INFO @ 07:31:36] Energy consumed for RAM : 0.000138 kWh. RAM Power : 4.754395008087158 W
[codecarbon INFO @ 07:31:36] Energy consumed for all GPUs : 0.001054 kWh. Total GPU Power : 61.577 W
[codecarbon INFO @ 07:31:36] En

0.01115044767594859

### BERT Large Cased

In [3]:
gc.collect()

60

In [4]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

# Compute intensive code goes here

ds = load_dataset("imdb")
model = AutoModelForSequenceClassification.from_pretrained("bert-large-cased",\
                                                           num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("bert-large-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

small_train_dataset = ds["train"].shuffle(seed=42).select(range(1000)).map(tokenize_function, batched=True)
small_eval_dataset = ds["test"].shuffle(seed=42).select(range(1000)).map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="/content/code-carbon-classification",
    num_train_epochs=1,
    push_to_hub=False,
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset
)

trainer.train()

# Compute intensive code stops here

tracker.stop()

[codecarbon INFO @ 08:06:56] [setup] RAM Tracking...
[codecarbon INFO @ 08:06:56] [setup] GPU Tracking...
[codecarbon INFO @ 08:06:56] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 08:06:56] [setup] CPU Tracking...
[codecarbon WARNING @ 08:06:56] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 08:06:58] We saw that you have a Intel(R) Xeon(R) CPU @ 2.30GHz but we don't know it. Please contact us.
[codecarbon INFO @ 08:06:58] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 08:06:58] >>> Tracker's metadata:
[codecarbon INFO @ 08:06:58]   Platform system: Linux-5.15.109+-x86_64-with-glibc2.35
[codecarbon INFO @ 08:06:58]   Python version: 3.10.6
[codecarbon INFO @ 08:06:58]   CodeCarbon version: 2.2.7
[codecarbon INFO @ 08:06:58]   Available RAM : 25.451 GB
[codecarbon INFO @ 08:06:58]   CPU count: 4
[codecarbon INFO @ 08:06:58]   CPU model: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 08:06:58]   GPU 

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

[codecarbon INFO @ 08:07:13] Energy consumed for RAM : 0.000040 kWh. RAM Power : 9.544258117675783 W
[codecarbon INFO @ 08:07:13] Energy consumed for all GPUs : 0.000107 kWh. Total GPU Power : 25.699 W
[codecarbon INFO @ 08:07:13] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:07:13] 0.000324 kWh of electricity used since the beginning.


Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

[codecarbon INFO @ 08:07:43] Energy consumed for RAM : 0.000119 kWh. RAM Power : 9.544258117675783 W
[codecarbon INFO @ 08:07:43] Energy consumed for all GPUs : 0.000389 kWh. Total GPU Power : 42.228 W
[codecarbon INFO @ 08:07:43] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 08:07:43] 0.001040 kWh of electricity used since the beginning.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


OutOfMemoryError: ignored